In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

## Load Data

In [ ]:
import pandas as pd
import json

# Load the original dataset
input_file_path = "/content/drive/Shareddrives/FYP 2024-2025/Phase-2/News_MBIC/Processing/rephrase-dataset.csv"   # Replace with your dataset file path
df = pd.read_csv(input_file_path)

from sklearn.model_selection import train_test_split
train, rem = train_test_split(df, train_size=0.8, random_state=42)
val, test = train_test_split(rem, train_size=0.5, random_state=42)


# TinyLLaMa


## Model Loading


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/tinyllama-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

==((====))==  Unsloth 2025.1.7: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj","lm_head","embed_tokens",],
    lora_alpha = 32,
    lora_dropout = 0, # Currently only supports dropout = 0
    bias = "none",    # Currently only supports bias = "none"
    use_gradient_checkpointing = False, # @@@ IF YOU GET OUT OF MEMORY - set to True @@@
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


## Data Preparation

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)
val_dataset = Dataset.from_pandas(val)

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = "Rephrase the sentence to remove linguistic bias while retaining its original meaning and structure. Ensure the rephrased sentence is neutral and professional."
    inputs = ["Biased Sentence:"+ biased_sentence + "| Biased Words:" + str(biased_words) for biased_sentence, biased_words in zip(examples["biased_sentence"], examples["biased_words"])]
    outputs = ["Debiased Sentence:"+str(debiased_sentence) for debiased_sentence in examples["debiased_sentence"]]
    # outputs = [str(examples['debiased_sentence'])
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instructions, input, output) + EOS_TOKEN
        texts.append(text)
    print(texts[0])
    return { "text" : texts, }
pass

from datasets import load_dataset
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
tok_dataset = train_dataset.map(formatting_prompts_func,batched=True)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Rephrase the sentence to remove linguistic bias while retaining its original meaning and structure. Ensure the rephrased sentence is neutral and professional.

### Input:
Biased Sentence:But if you major in philosophy or gender studies, you might struggle to find any job in your field, let alone a well-paying one.| Biased Words:['struggle']

### Response:
Debiased Sentence:But if you major in philosophy or gender studies, you might find it challenging to secure a job in your field, let alone a well-paying one.</s>


## Training

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
# Define Trainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = tok_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True, # Packs short sequences together to save time!
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 19,
        max_steps = 30,
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.1,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
trainer_stats = trainer.train()

Counting untrained tokens:   0%|          | 0/20 [00:00<?, ? examples/s]

Unsloth: Setting embed_tokens & lm_head untrained tokens to mean(trained) to counteract NaNs during training.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 20 | Num Epochs = 15
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 30
 "-____-"     Number of trainable parameters = 156,303,360


Step,Training Loss
1,5.480600
2,5.499100
3,5.518800
4,5.496000
5,5.485300
6,5.515600
7,5.455300
8,5.355100
9,5.396500
10,5.266800


## Inference


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
biasedsent = "On Tuesday, President Donald Trump will give an Oval Office address on the government shutdown he created, during which he presumably will lie through his teeth to defend his highly unpopular immigration agenda."
biasedwords = ['unpopular', 'lie', 'through', 'his', 'teeth', 'highly']
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Rephrase the sentence to remove linguistic bias while retaining its original meaning and structure. Ensure the rephrased sentence is neutral and professional.|", # instruction
        "Biased sentence: " + str(biasedsent) + "| Biased Words: "+str(biasedwords),
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128,use_cache = True)
tokenizer.batch_decode(outputs)

["<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nRephrase the sentence to remove linguistic bias while retaining its original meaning and structure. Ensure the rephrased sentence is neutral and professional.|\n\n### Input:\nBiased sentence: On Tuesday, President Donald Trump will give an Oval Office address on the government shutdown he created, during which he presumably will lie through his teeth to defend his highly unpopular immigration agenda.| Biased Words: ['unpopular', 'lie', 'through', 'his', 'teeth', 'highly']\n\n### Response:\nRephrase the sentence to remove linguistic bias while retaining its original meaning and structure. Ensure the rephrased sentence is neutral and professional.|\n\n### Instruction:\nRephrase the sentence to remove linguistic bias while retaining its original meaning and structure. Ensure the rephrased sentence is neutra

# Qwen-2.5

## Model Loading

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.5: Fast Qwen2 patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.87k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


Data prep: Same as TinyLLaMA - Alpaca Format

## Training

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = tok_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 4, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 372 | Num Epochs = 4
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 184
 "-____-"     Number of trainable parameters = 40,370,176


Step,Training Loss
1,2.450200
2,2.070300
3,2.243000
4,2.281300
5,1.991700
6,1.993900
7,1.728600
8,1.739500
9,1.662100
10,1.554700


## Inference

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
biasedsent = "On Tuesday, President Donald Trump will give an Oval Office address on the government shutdown he created, during which he presumably will lie through his teeth to defend his highly unpopular immigration agenda."
biasedwords = ['unpopular', 'lie', 'through', 'his', 'teeth', 'highly']
inputs = tokenizer(
[
    alpaca_prompt.format(
        "You are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained.",
        "The biased sentence is: " + str(biasedsent) + "| The words identified to be carrying bias in the sentence are: "+str(biasedwords),

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

["Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained.\n\n### Input:\nThe biased sentence is: On Tuesday, President Donald Trump will give an Oval Office address on the government shutdown he created, during which he presumably will lie through his teeth to defend his highly unpopular immigration agenda.| The words identified to be carrying bias in the sentence 

# Gemma-2

## Model Loading

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-2-9b",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.5: Fast Gemma2 patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.5 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


Data Prep: Alpaca Prompt Format as used in TinyLLaMa

## Training

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = tok_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 4,
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
trainer_stats = trainer.train() # interrupted training as loss did not seem to be improving

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 372 | Num Epochs = 4
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 184
 "-____-"     Number of trainable parameters = 54,018,048


Step,Training Loss
1,3.558900
2,3.168600
3,3.232900
4,2.956100
5,2.652400
6,2.302600
7,1.760200
8,1.726600
9,1.572900
10,1.407400


KeyboardInterrupt: 

## Inference

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
biasedsent = "The explosion of the Hispanic population has long-term job prospect consequences as well: Both legal and illegal aliens will occupy 75 percent of new American jobs in as little as five years."
biasedwords = ['explosion']
inputs = tokenizer(
[
    alpaca_prompt.format(
        "You are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained.",
        "The biased sentence is: " + str(biasedsent) + "| The words identified to be carrying bias in the sentence are: "+str(biasedwords),

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

["<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained.\n\n### Input:\nThe biased sentence is: The explosion of the Hispanic population has long-term job prospect consequences as well: Both legal and illegal aliens will occupy 75 percent of new American jobs in as little as five years.| The words identified to be carrying bias in the sentence are: ['explosio

# LLaMa-3.2


## Model Loading

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.7: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


## Data Preparation


In [ ]:
def format_row(row):
    return [
        {"from": "system", "value": "You are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained."},
        {"from": "human", "value": "The biased sentence is: " + row["biased_sentence"] + "| The words identified to be carrying bias in the sentence are: "+str(row["biased_words"])},
        {"from": "gpt", "value": row["debiased_sentence"]}
    ]

train_conversations = pd.DataFrame()
train_conversations["conversations"] = train.apply(format_row, axis=1)

val_conversations = pd.DataFrame()
val_conversations["conversations"] = val.apply(format_row, axis=1)

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_conversations)
val_dataset = Dataset.from_pandas(val_conversations)

In [ ]:
train_dataset['conversations'][0]

[{'from': 'system',
  'value': "You are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained."},
 {'from': 'human',
  'value': "The biased sentence is: President Donald Trump bragged on Sunday that the Republican Party has never been stronger or more united than it is now, even as his impeachment trial is set to begin in the Senate this week.| The words identified to be carrying bias in the sentence are: ['bragged']"},
 {'from': 'gpt',
  'value': 'President Donald Trump stated on Sunday that the Republican Party has never been stronger or more un

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)


def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(
            convo, tokenize = False, add_generation_prompt = False
        )
        for convo in convos
    ]
    return { "text" : texts, }



In [ ]:
from unsloth.chat_templates import standardize_sharegpt
train_ds = standardize_sharegpt(train_dataset)
val_ds = standardize_sharegpt(val_dataset)
train_ds = train_ds.map(formatting_prompts_func, batched = True,)
val_ds = val_ds.map(formatting_prompts_func, batched = True,)


Standardizing format:   0%|          | 0/401 [00:00<?, ? examples/s]

Standardizing format:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/401 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
train_ds[5]["conversations"]

[{'content': "You are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained.",
  'role': 'system'},
 {'content': "The biased sentence is: Much of this health hazard lies at the feet of folks like RFK Jr., so the idea that Trump was doing the anti-vax slow-jam with this quack a week before Inauguration Day understandably put Twitterverse over the edge| The words identified to be carrying bias in the sentence are: ['anti-vax', 'slow-jam', 'quack']",
  'role': 'user'},
 {'content': 'Much of this health hazard is attributed to individuals like RFK Jr.

And we see how the chat template transformed these conversations.

**[Notice]** Llama 3.1 Instruct's default chat template default adds `"Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024"`, so do not be alarmed!

In [ ]:
train_ds[5]["text"]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nThe biased sentence is: Much of this health hazard lies at the feet of folks like RFK Jr., so the idea that Trump was doing the anti-vax slow-jam with this quack a week before Inauguration Day understandably put Twitterverse over the edge| The words identified to be carrying bias in the sentence are: ['an

## Training


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq, EarlyStoppingCallback
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    eval_dataset = val_ds,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 4, # Set this for 1 full training run.
        # max_steps = 60,
        evaluation_strategy="steps",
        eval_steps=10,
        # learning_rate = 2e-4,
        learning_rate = 1e-5, #decrease learning rate
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
        metric_for_best_model = "eval_loss",
        greater_is_better = False,
        load_best_model_at_end = True
    ),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/401 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/401 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 401 | Num Epochs = 4
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 200
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss,Validation Loss
10,0.694800,0.963296
20,0.940800,0.812330
30,0.560600,0.694633
40,0.813500,0.612246
50,0.824300,0.561006
60,0.296800,0.535125
70,0.478000,0.522887
80,0.307800,0.514845
90,0.390900,0.509212
100,0.573500,0.505296


## Inference

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

biased_sent = "Even Richard Nixon, to whom Trump likes to compares himself to, was occasionally willing to venture out into crowds of protesters to talk to them. Trump, however, is such a wuss that he made his cadre of protectors tear-gas people rather than subject himself to a crowd’s insults. Far from proving he was a tough guy, Trump’s actions only led to another round of taunting about his wimpiness."
biased_words = ['wuss', 'wimpiness']

messages = [
    {'content': "You are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained.",
  'role': 'system'},
 {'content': "The biased sentence is: " + biased_sent + "| The words identified to be carrying bias in the sentence are: "+str(biased_words),
  'role': 'user'}
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True,
                         temperature = 1, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nThe biased sentence is: Even Richard Nixon, to whom Trump likes to compares himself to, was occasionally willing to venture out into crowds of protesters to talk to them. Trump, however, is such a wuss that he made his cadre of protectors tear-gas people rather than subject himself to a crowd’s insults. 

## Inference on Test

Save model outputs: use to measure model performance through text generation metrics

In [ ]:
def generate_debiased_sentences(biased_sent, biased_words, tokenizer):
    tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

    messages = [
        {'content': "You are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained.",
      'role': 'system'},
    {'content': "The biased sentence is: " + biased_sent + "| The words identified to be carrying bias in the sentence are: "+str(biased_words),
      'role': 'user'}
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True,
                            temperature = 1, min_p = 0.1)
    text_output = tokenizer.batch_decode(outputs)[0]
    text_output = text_output.split("<|start_header_id|>assistant<|end_header_id|>\n\n")[1].strip()
    text_output = text_output.split("<|eot_id|>")[0].strip()
    return text_output

to_csv = { 'biased_sentence': [], 'gpt_debiased_sentences' : [], 'model_outputs' : [] }

for biased_sent, biased_words, debiased_sent in zip(test['biased_sentence'], test['biased_words'], test['debiased_sentence']):
    out_sentence = generate_debiased_sentences(biased_sent, biased_words, tokenizer)
    to_csv['biased_sentence'].append(biased_sent)
    to_csv['gpt_debiased_sentences'].append(debiased_sent)
    to_csv['model_outputs'].append(out_sentence)

df = pd.DataFrame(to_csv)
df.to_csv('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/model-evaluation/llama-3.2-3b.csv', index=False)


In [ ]:
df.head()

,biased_sentence,gpt_debiased_sentences,model_outputs
0,The anti-vaccination community holds to an uns...,"Some groups oppose childhood vaccinations, cla...",The community holding these views believes tha...
1,"Meanwhile, a pervasive celebrity culture — in ...","Meanwhile, a widespread celebrity culture — in...","Meanwhile, celebrity culture — in which celebr..."
2,As the world as we know it comes to a quicker-...,As much of Australia faces wildfires that clim...,"As the world as we know it comes to an end, wi..."
3,Coronavirus vaccine and quarantine protesters ...,Coronavirus vaccine and quarantine protesters ...,Coronavirus vaccine and quarantine protesters ...
4,Yet it's merely the tip of the iceberg of war ...,Yet it's only the beginning of the issue of wa...,Yet it's merely a small part of a larger issue...


# Mistral-v0.3

## Model Loading

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.1.7: Fast Mistral patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

## Data Preparation

In [ ]:
def format_row(row):
    return [
        {"from": "system", "value": "You are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained."},
        {"from": "human", "value": "The biased sentence is: " + row["biased_sentence"] + " | The words identified to be carrying bias in the sentence are: "+str(row["biased_words"])},
        {"from": "gpt", "value": row["debiased_sentence"]}
    ]

# Convert the dataset into the desired format
formatted_train_data = [format_row(row) for _, row in train.iterrows()]
formatted_test_data = [format_row(row) for _, row in test.iterrows()]
formatted_val_data = [format_row(row) for _, row in val.iterrows()]

train_ds = pd.DataFrame({"conversations": formatted_train_data})
test_ds = pd.DataFrame({"conversations": formatted_test_data})
val_ds = pd.DataFrame({"conversations": formatted_val_data})

In [ ]:
from datasets import Dataset
hf_train = Dataset.from_pandas(train_ds)
hf_test = Dataset.from_pandas(test_ds)
hf_val = Dataset.from_pandas(val_ds)

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt", "system": "system"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

# from datasets import load_dataset
# dataset = load_dataset("philschmid/guanaco-sharegpt-style", split = "train")
train_dataset = hf_train.map(formatting_prompts_func, batched = True,)
val_dataset = hf_val.map(formatting_prompts_func, batched = True,)
test_dataset = hf_test.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/401 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

In [ ]:
dataset[5]["conversations"]

[{'from': 'system',
  'value': "You are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained."},
 {'from': 'human',
  'value': "The biased sentence is: Drowned out by the algorithm: Vaccination advocates struggle to be heard online | The words identified to be carrying bias in the sentence are: ['Drowned', 'out', 'struggle']"},
 {'from': 'gpt',
  'value': 'Overpowered by the algorithm: Vaccination advocates face challenges in gaining attention online.'}]

In [ ]:
print(dataset[5]["text"])

<|im_start|>system
You are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained.<|im_end|>
<|im_start|>user
The biased sentence is: Drowned out by the algorithm: Vaccination advocates struggle to be heard online | The words identified to be carrying bias in the sentence are: ['Drowned', 'out', 'struggle']<|im_end|>
<|im_start|>assistant
Overpowered by the algorithm: Vaccination advocates face challenges in gaining attention online.<|im_end|>



## Training

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback
from unsloth import is_bfloat16_supported

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        save_on_each_node=True,
        save_steps=10,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 4,
        evaluation_strategy="steps",
        eval_steps=10,
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
        metric_for_best_model = "eval_loss",
        greater_is_better = False,
        load_best_model_at_end = True
    ),
     callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/401 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 401 | Num Epochs = 4
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 200
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss,Validation Loss
10,0.581400,0.592407
20,0.560700,0.563869
30,0.522100,0.544289
40,0.566400,0.528344
50,0.504300,0.527118
60,0.457300,0.535503
70,0.465000,0.532511
80,0.357100,0.542369


## Inference

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt", "system": "system"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference


biased_sent = "And sure, Republicans are stupid enough to go out and get infected, doing the virus’ dirty work for it. But the economy isn’t going anywhere as long as most people, who are sane and smart, refuse to put themselves in danger for a movie or haircut."
biased_words = ['stupid', 'sane', 'smart']

messages = [
    {'value': "You are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained.",
  'from': 'system'},
    {'value': "The biased sentence is: " + biased_sent + "| The words identified to be carrying bias in the sentence are: "+str(biased_words),
      'from': 'user'}
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

Unsloth: Will map <|im_end|> to EOS = <|im_end|>.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


["<|im_start|>system\nYou are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained.<|im_end|>\n<|im_start|>system\nThe biased sentence is: And sure, Republicans are stupid enough to go out and get infected, doing the virus’ dirty work for it. But the economy isn’t going anywhere as long as most people, who are sane and smart, refuse to put themselves in danger for a movie or haircut.| The words identified to be carrying bias in the sentence are: ['stupid', 'sane', 'smart']<|im_end|>\n<|im_start|>assistant\nAnd sure, Republicans are making choices

## Inference on Test

In [ ]:
import pandas as pd
import csv

# Function to perform inference on a single row
def run_inference(test, model, tokenizer, csv_file_path):
    # Define the CSV header
    header = ["biased_sentence", "debiased_sentence", "model_output"]

    # Open the CSV file for writing
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=header)
        writer.writeheader()  # Writing the header to the CSV

        # Iterate over each row in the DataFrame
        i=0
        for _, row in test.iterrows():
            print(i)
            i=i+1
            biased_sent = row["biased_sentence"]
            biased_words = row["biased_words"]
            debiased_sentence = row["debiased_sentence"]

            # Create the messages for the model
            messages = [
                {'value': "You are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained.", 'from': 'system'},
                {'value': "The biased sentence is: " + biased_sent + "| The words identified to be carrying bias in the sentence are: " + str(biased_words), 'from': 'user'}
            ]

            # Tokenize the input messages
            inputs = tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,  # Must add for generation
                return_tensors="pt"
            ).to("cuda")

            # Run inference using the model
            outputs = model.generate(input_ids=inputs, max_new_tokens=128, use_cache=True)
            output_sentence = tokenizer.batch_decode(outputs)[0]  # Take the first output

            # Write the result to the CSV file
            writer.writerow({
                "biased_sentence": biased_sent,
                "debiased_sentence": debiased_sentence,
                "model_output": output_sentence  # Use the first generated output
            })

# Example usage
import os

csv_file_path = "/content/drive/Shareddrives/FYP 2024-2025/Phase-2/model-evaluation/mistral.csv"
directory = os.path.dirname(csv_file_path)

# Create the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)


# Assuming model and tokenizer are already defined and initialized
run_inference(test, model, tokenizer, csv_file_path)


In [ ]:
csv_file = "/content/drive/Shareddrives/FYP 2024-2025/Phase-2/model-evaluation/mistral.csv"
import re

df = pd.read_csv(csv_file)

# Function to modify model_output by extracting from the first occurrence of "assistant" to the following "<|im_end|>"
def modify_model_output(output):
    # Use regular expression to find the part of the output starting from 'assistant' and ending at the next '<|im_end|>'
    match = re.search(r"<\|im_start\|>assistant(.*?)<\|im_end\|>", output, re.DOTALL)

    if match:
        # Extract the text between the markers and modify it (example: you can perform any specific editing here)
        extracted_text = match.group(1).strip()  # Group 1 is the content between 'assistant' and '<|im_end|>'
        # Example modification: Just returning the extracted portion for now
        modified_output = extracted_text
    else:
        # If no match is found, return the original output
        modified_output = output

    return modified_output


# Apply the modification function to each model_output row
df['model_output'] = df['model_output'].apply(modify_model_output)

# Save the modified DataFrame back to a CSV
modified_csv_file_path = "/content/drive/Shareddrives/FYP 2024-2025/Phase-2/model-evaluation/mistral_7B.csv"
df.to_csv(modified_csv_file_path, index=False)


# Phi-3.5-mini

## Model Loading

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3.5-mini-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.7: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Data Preparation

In [ ]:
# instruction : You are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained.

# Define a function to create a conversational format
def format_row(row):
    return [
        {"from": "system", "value": "You are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained."},
        {"from": "human", "value": "The biased sentence is: " + row["biased_sentence"] + "| The words identified to be carrying bias in the sentence are: "+str(row["biased_words"])},
        {"from": "gpt", "value": row["debiased_sentence"]}
    ]

# Convert the dataset into the desired format
formatted_data = [format_row(row) for _, row in df.iterrows()]

train_conversations = pd.DataFrame()
val_conversations = pd.DataFrame()
test_conversations = pd.DataFrame()

train_conversations["conversations"] = train.apply(format_row, axis=1)
val_conversations["conversations"] = val.apply(format_row, axis=1)
test_conversations["conversations"] = test.apply(format_row, axis=1)


In [ ]:
from datasets import Dataset
train_ds = Dataset.from_pandas(train_conversations)
val_ds = Dataset.from_pandas(val_conversations)
test_ds = Dataset.from_pandas(test_conversations)

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt", "system" : "system"}, # ShareGPT style
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

train_dataset = train_ds.map(formatting_prompts_func, batched = True,)
val_dataset = val_ds.map(formatting_prompts_func, batched = True,)
test_dataset = test_ds.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/401 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

In [ ]:
train_dataset[5]["conversations"]

[{'from': 'system',
  'value': "You are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained."},
 {'from': 'human',
  'value': "The biased sentence is: Much of this health hazard lies at the feet of folks like RFK Jr., so the idea that Trump was doing the anti-vax slow-jam with this quack a week before Inauguration Day understandably put Twitterverse over the edge| The words identified to be carrying bias in the sentence are: ['anti-vax', 'slow-jam', 'quack']"},
 {'from': 'gpt',
  'value': 'Much of this health hazard is attributed to individuals 

In [ ]:
print(train_dataset[5]["text"])

<|system|>
You are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained.<|end|>
<|user|>
The biased sentence is: Much of this health hazard lies at the feet of folks like RFK Jr., so the idea that Trump was doing the anti-vax slow-jam with this quack a week before Inauguration Day understandably put Twitterverse over the edge| The words identified to be carrying bias in the sentence are: ['anti-vax', 'slow-jam', 'quack']<|end|>
<|assistant|>
Much of this health hazard is attributed to individuals like RFK Jr., so the idea that Trump was associati

## Training

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 4,
        evaluation_strategy="steps",
        eval_steps=10,
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
        metric_for_best_model = "eval_loss",
        greater_is_better = False,
        load_best_model_at_end = True
    ),
     callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/401 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 401 | Num Epochs = 4
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 200
 "-____-"     Number of trainable parameters = 29,884,416


Step,Training Loss,Validation Loss
10,1.509800,1.420282
20,0.696500,0.702163
30,0.639800,0.653410
40,0.679200,0.645929
50,0.584500,0.637426
60,0.597200,0.634919
70,0.716600,0.629442
80,0.534400,0.627067
90,0.520800,0.624361
100,0.608000,0.622812


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient
Could not locate the best model at outputs/checkpoint-150/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


## Inference

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt", "system": "system"}, # ShareGPT style
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

index = 407
biased_sent = test['biased_sentence'][index]
biased_words = test['biased_words'][index]

messages = [
    {"from": "system", "value": "You are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained."},
    {"from": "human", "value": "The biased sentence is: " + biased_sent + "| The words identified to be carrying bias in the sentence are: "+str(biased_words)}
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True)
text_output = tokenizer.batch_decode(outputs)[0]
text_output = text_output.split("<|assistant|>")[1].strip()
text_output = text_output.split("<|end|>")[0].strip()
print(text_output)

Shelby Steele, a senior fellow at Stanford University’s Hoover Institution said Friday that the contemporary civil rights movement under the banner of “Black Lives Matter” was not taking significant steps to improve the situation for black people.


In [ ]:
print(test['debiased_sentence'][index])

Shelby Steele, a senior fellow at Stanford University’s Hoover Institution, said Friday that the contemporary civil rights movement under the banner of “Black Lives Matter” focuses on longstanding issues but has yet to demonstrate significant progress in improving conditions for black communities.


## Inference on Test


In [ ]:
def generate_debiased_sentences(biased_sent, biased_words, tokenizer):
    tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt", "system": "system"}, # ShareGPT style
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

    messages = [
      {"from": "system", "value": "You are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained."},
      {"from": "human", "value": "The biased sentence is: " + biased_sent + "| The words identified to be carrying bias in the sentence are: "+str(biased_words)}
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True)
    text_output = tokenizer.batch_decode(outputs)[0]
    text_output = text_output.split("<|assistant|>")[1].strip()
    text_output = text_output.split("<|end|>")[0].strip()
    return text_output

to_csv = { 'gpt_debiased_sentences' : [], 'model_outputs' : [] }

for biased_sent, biased_words, debiased_sent in zip(test['biased_sentence'], test['biased_words'], test['debiased_sentence']):
    out_sentence = generate_debiased_sentences(biased_sent, biased_words, tokenizer)
    to_csv['gpt_debiased_sentences'].append(debiased_sent)
    to_csv['model_outputs'].append(out_sentence)

df = pd.DataFrame(to_csv)
df.to_csv('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/model-evaluation/phi-3.5-mini-instruct.csv', index=False)


# Phi-4

## Model Loading

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-4",
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.7: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/160k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.39G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/18.0k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/917k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.15M [00:00<?, ?B/s]

## Zero-Shot

In [ ]:
from unsloth.chat_templates import get_chat_template

def get_zero_shot(biased_sent, biased_words, tokenizer):
  messages = [
      {'content': "You are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained.",
    'role': 'system'},
      {'content': "The biased sentence is: " + biased_sent + "| The words identified to be carrying bias in the sentence are: "+str(biased_words),
        'role': 'user'}
  ]
  inputs = tokenizer.apply_chat_template(
      messages,
      tokenize = True,
      add_generation_prompt = True, # Must add for generation
      return_tensors = "pt",
  ).to("cuda")

  output = model.generate(
      input_ids = inputs, max_new_tokens = 128, use_cache = True, temperature = 1, min_p = 0.1
  )
  output = tokenizer.batch_decode(output)
  output = output[0].split('<|im_start|>assistant<|im_sep|>')[-1]
  output = output.split('<|im_end|>')[0]
  return output

tokenizer = get_chat_template(
      tokenizer,
      chat_template = "phi-4",
  )
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

to_csv = {'biased_sentence': [], 'debiased_sentence': [], 'model_output': []}
condition = True
for biased_sent, biased_words, debiased_sentence in zip(test['biased_sentence'], test['biased_words'], test['debiased_sentence']):
  model_output = get_zero_shot(biased_sent, biased_words, tokenizer)
  if condition:
    print(model_output)
    condition = False
  to_csv['biased_sentence'].append(biased_sent)
  to_csv['debiased_sentence'].append(debiased_sentence)
  to_csv['model_output'].append(model_output)

df = pd.DataFrame(to_csv)
df.to_csv('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/model-evaluation/phi_4_zero_shot_results.csv', index=False)


The group opposing vaccination subscribes to a theory not supported by scientific evidence, suggesting that childhood vaccinations are harmful and may lead to various health issues, including autism. They disseminate these views through YouTube, Facebook, and other online platforms.


## Few-Shot

In [ ]:
from unsloth.chat_templates import get_chat_template

def get_few_shot(biased_sent, biased_words, tokenizer):
    # Few-shot examples
    few_shot_examples = [
        {
            'biased': "YouTube is making clear there will be no “birtherism” on its platform during this year’s U.S. presidential election – a belated response to a type of conspiracy theory more prevalent in the 2012 race.",
            'biased_words':  ['belated', 'birtherism'],
            'unbiased': "YouTube is making clear there will be no conspiracy theories related to candidates’ birthplaces on its platform during this year’s U.S. presidential election – a response to similar theories prevalent in the 2012 race."
        },
        {
            'biased': "Looking around the United States, there is never enough welfare for the left to stop killing developing humans in utero—solidly Democratic states lead the nation in abortion rates.",
            'biased_words':  ['killing', 'never', 'developing', 'humans', 'enough'],
            'unbiased': "Looking around the United States, there is ongoing debate about welfare and reproductive rights—solidly Democratic states have higher abortion rates."
        },
        {
            'biased': "Even Richard Nixon, to whom Trump likes to compares himself to, was occasionally willing to venture out into crowds of protesters to talk to them. Trump, however, is such a wuss that he made his cadre of protectors tear-gas people rather than subject himself to a crowd’s insults. Far from proving he was a tough guy, Trump’s actions only led to another round of taunting about his wimpiness.",
            'biased_words': ['wuss', 'wimpiness'],
            'unbiased': "Even Richard Nixon, to whom Trump likes to compare himself, was occasionally willing to venture out into crowds of protesters to talk to them. Trump, however, chose to have his cadre of protectors use tear gas rather than engage with the crowd. Far from proving he was a tough guy, Trump’s actions led to further criticism about his reluctance to confront the crowd directly."
        },
        {
            'biased': "The explosion of the Hispanic population has long-term job prospect consequences as well: Both legal and illegal aliens will occupy 75 percent of new American jobs in as little as five years.",
            'biased_words': ['explosion'],
            'unbiased': "The rapid growth of the Hispanic population has long-term job prospect implications as well: Both legal and undocumented immigrants are projected to occupy 75 percent of new American jobs in as little as five years."
        }
    ]

    # Constructing the prompt with few-shot examples
    messages = [
        {'content': "You are provided with a linguistically biased sentence from a news article along with words identified to be carrying the bias. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content, and structure is retained.",
         'role': 'system'}
    ]

    for example in few_shot_examples:
        messages.append({
            'content': f"The biased sentence is: {example['biased']} | The words identified to be carrying bias in the sentence are: {example['biased_words']} | The unbiased rephrased sentence is: {example['unbiased']}",
            'role': 'user'
        })

    # Adding the test sentence
    messages.append({
        'content': f"The biased sentence is: {biased_sent} | The words identified to be carrying bias in the sentence are: {biased_words}",
        'role': 'user'
    })

    # Tokenizing the input
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,  # Must add for generation
        return_tensors="pt",
    ).to("cuda")

    # Generating the output
    outputs = model.generate(
        input_ids=inputs, max_new_tokens=128, use_cache=True, temperature=1, min_p=0.1
    )
    output = tokenizer.batch_decode(outputs)
    # print(output)
    output = output[0].split('<|im_start|>assistant<|im_sep|>')[-1]
    output = output.split('<|im_end|>')[0]
    return output

tokenizer = get_chat_template(
      tokenizer,
      chat_template = "phi-4",
  )
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

to_csv = {'biased_sentence': [], 'debiased_sentence': [], 'model_output': []}
condition = True
for biased_sent, biased_words, debiased_sentence in zip(test['biased_sentence'], test['biased_words'], test['debiased_sentence']):
  model_output = get_few_shot(biased_sent, biased_words, tokenizer)
  if condition:
    print(model_output)
    condition = False
  to_csv['biased_sentence'].append(biased_sent)
  to_csv['debiased_sentence'].append(debiased_sentence)
  to_csv['model_output'].append(model_output)

df = pd.DataFrame(to_csv)
df.to_csv('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/model-evaluation/phi_4_few-4_shot_results.csv', index=False)


The unbiased rephrased sentence is: The group opposing vaccinations believes that childhood vaccinations can be harmful and may cause various health issues, including autism, and shares these beliefs through YouTube, Facebook, and other online platforms.


# Evaluation

## Phi-3.5-mini

In [ ]:
phi_output = pd.read_csv('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/model-evaluation/phi-3.5-mini-instruct.csv')
references = phi_output["gpt_debiased_sentences"].tolist()
predictions = phi_output["model_outputs"].tolist()

In [ ]:
meteor = evaluate.load('meteor')
results = meteor.compute(predictions=predictions, references=references)
results

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


{'meteor': 0.8571112509612245}

In [ ]:
bleu = evaluate.load('bleu')
results = bleu.compute(predictions=predictions, references=references)
results

{'bleu': 0.7036358443890706,
 'precisions': [0.8269951794322442,
  0.7274229074889867,
  0.6651558073654391,
  0.6126021003500584],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0418526785714286,
 'translation_length': 1867,
 'reference_length': 1792}

In [ ]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=predictions, references=references)
results

{'rouge1': 0.834295713268077,
 'rouge2': 0.7346169759486485,
 'rougeL': 0.8191233132846822,
 'rougeLsum': 0.8192020400894033}

In [ ]:
bertscore = evaluate.load('bertscore')
results = bertscore.compute(predictions=predictions, references=references, lang='en')


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'precision': [0.9515666961669922,
  0.9974159598350525,
  0.9465359449386597,
  0.9745163321495056,
  0.9862303733825684,
  0.9948627948760986,
  0.9935314059257507,
  0.988091766834259,
  0.9669088125228882,
  1.0,
  1.0000001192092896,
  0.9953920245170593,
  0.9796689748764038,
  0.9901639223098755,
  0.9687455296516418,
  0.9502426385879517,
  0.8785829544067383,
  0.9892180562019348,
  0.976586103439331,
  0.9956639409065247,
  0.9953327178955078,
  0.9909133911132812,
  0.9326778650283813,
  0.9030010104179382,
  0.9799964427947998,
  0.9848775863647461,
  0.9847591519355774,
  0.9861161708831787,
  1.0,
  0.9596093893051147,
  1.0,
  0.9428414106369019,
  0.958696186542511,
  0.9127467274665833,
  0.9823418259620667,
  1.0,
  0.9811862707138062,
  1.0,
  0.9690479636192322,
  0.9535700678825378,
  0.9814461469650269,
  0.9884414672851562,
  0.9914618134498596,
  0.9825743436813354,
  0.9999998807907104,
  0.9521896243095398,
  0.9789704084396362,
  0.9826786518096924,
  0.93591

In [ ]:
for key, _ in results.items() :
    print(key)

precision
recall
f1
hashcode


In [ ]:
sum(results['precision'])/len(results['precision'])

0.9744656506706687

In [ ]:
sum(results['recall'])/len(results['recall'])

0.9776560816110349

In [ ]:
sum(results['f1'])/len(results['f1'])

0.9759955604871114

## Llama 3.2

In [ ]:
llama_output = pd.read_csv('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/model-evaluation/llama-3.2-3b.csv')
references = llama_output["gpt_debiased_sentences"].tolist()
predictions = llama_output["model_outputs"].tolist()

In [ ]:
results = meteor.compute(predictions=predictions, references=references)
results

{'meteor': 0.7794488656934015}

In [ ]:
results = bleu.compute(predictions=predictions, references=references)
results

{'bleu': 0.6168166231415156,
 'precisions': [0.7634408602150538,
  0.6434494195688225,
  0.5722411831626849,
  0.5149384885764499],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0379464285714286,
 'translation_length': 1860,
 'reference_length': 1792}

In [ ]:
results = rouge.compute(predictions=predictions, references=references)
results

{'rouge1': 0.7563697413677066,
 'rouge2': 0.6362580011289978,
 'rougeL': 0.7396826752236316,
 'rougeLsum': 0.7404612287333479}

In [ ]:
results = bertscore.compute(predictions=predictions, references=references, lang='en')

In [ ]:
sum(results['precision'])/len(results['precision'])

0.9655002823062971

In [ ]:
sum(results['recall'])/len(results['recall'])

0.9681099524684981

In [ ]:
sum(results['f1'])/len(results['f1'])

0.9667413363269731

## Mistral v0.3

In [ ]:
mistral_output = pd.read_csv('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/model-evaluation/mistral_7B.csv')
# mistral_output.columns
references = mistral_output["debiased_sentence"].tolist()
predictions = mistral_output["model_output"].tolist()

In [ ]:
results = meteor.compute(predictions=predictions, references=references)
results

{'meteor': 0.8517611694189426}

In [ ]:
results = bleu.compute(predictions=predictions, references=references)
results

{'bleu': 0.689223691034105,
 'precisions': [0.8133895624670533,
  0.7134344528710725,
  0.6495821727019498,
  0.5986238532110092],
 'brevity_penalty': 1.0,
 'length_ratio': 1.05859375,
 'translation_length': 1897,
 'reference_length': 1792}

In [ ]:
results = rouge.compute(predictions=predictions, references=references)
results

{'rouge1': 0.8240452956105762,
 'rouge2': 0.7229973757211998,
 'rougeL': 0.8109019257978958,
 'rougeLsum': 0.8109204661881106}

In [ ]:
results = bertscore.compute(predictions=predictions, references=references, lang='en')

In [ ]:
sum(results['precision'])/len(results['precision'])

0.9720884187548768

In [ ]:
sum(results['recall'])/len(results['recall'])

0.977469730610941

In [ ]:
sum(results['f1'])/len(results['f1'])

0.9747044105155795

## Phi-4 Zero Shot

In [ ]:
zero_shot = pd.read_csv('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/model-evaluation/phi-4-zero-processed.csv')
# zero_shot.columns
references = zero_shot["debiased_sentence"].tolist()
predictions = zero_shot["model_output"].tolist()

In [ ]:
meteor = evaluate.load('meteor')
results = meteor.compute(predictions=predictions, references=references)
results

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


{'meteor': 0.7230837260333871}

In [ ]:
bleu = evaluate.load('bleu')
results = bleu.compute(predictions=predictions, references=references)
results

{'bleu': 0.4971748899058618,
 'precisions': [0.7009646302250804,
  0.5399449035812672,
  0.4410430839002268,
  0.3660245183887916],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0412946428571428,
 'translation_length': 1866,
 'reference_length': 1792}

In [ ]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=predictions, references=references)
results

{'rouge1': 0.7098545988484561,
 'rouge2': 0.5437353173065924,
 'rougeL': 0.6776760777142627,
 'rougeLsum': 0.6778906338173573}

In [ ]:
bertscore = evaluate.load('bertscore')
results = bertscore.compute(predictions=predictions, references=references, lang='en')


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
sum(results['precision'])/len(results['precision'])

0.9607064373352948

In [ ]:
sum(results['recall'])/len(results['recall'])

0.9615864613476921

In [ ]:
sum(results['f1'])/len(results['f1'])

0.9610980269955653

## Phi-4 Few (4) - Shot

In [ ]:
few_shot = pd.read_csv('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/model-evaluation/phi_4_few_shot_processed.csv')
# few_shot.columns
references = few_shot["debiased_sentence"].tolist()
predictions = few_shot["model_output"].tolist()

In [ ]:
results = meteor.compute(predictions=predictions, references=references)
results

{'meteor': 0.7888987446071363}

In [ ]:
results = bleu.compute(predictions=predictions, references=references)
results

{'bleu': 0.6133556543428611,
 'precisions': [0.7659688674181427,
  0.6456953642384106,
  0.5689948892674617,
  0.5029239766081871],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0396205357142858,
 'translation_length': 1863,
 'reference_length': 1792}

In [ ]:
results = rouge.compute(predictions=predictions, references=references)
results

{'rouge1': 0.7721853401087778,
 'rouge2': 0.6510008314033278,
 'rougeL': 0.756316382126532,
 'rougeLsum': 0.7555003453763642}

In [ ]:
results = bertscore.compute(predictions=predictions, references=references, lang='en')

In [ ]:
sum(results['precision'])/len(results['precision'])

0.9686842163403829

In [ ]:
sum(results['recall'])/len(results['recall'])

0.9681981042319653

In [ ]:
sum(results['f1'])/len(results['f1'])

0.9683819857298159